In [1]:
import math
import json
import random
import copy
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, metrics
import pandas as pd
import numpy as np
iris = datasets.load_iris()

X = iris['data']
y = iris['target']

# Combine the data and target arrays
data = np.hstack((X, y.reshape(-1, 1)))

# Shuffle the data randomly
np.random.shuffle(data)

# Separate the shuffled data and target arrays
X = data[:, :-1]
y = data[:, -1].astype(int)

In [2]:
def linear(net):
    return net

def relu(net):
    return max(0, net)

def sigmoid(net):
    try:
        return 1 / (1 + math.exp((-1) * net))
    except OverflowError:
        return 0

def softmax(net):
    try:
        total_softmax = 0
        for i in range(len(net)):
            net[i] = math.exp(net[i])
            total_softmax += net[i]
        for i in range(len(net)):
            net[i] = net[i] / total_softmax
    except OverflowError: 
        for i in range(len(net)):
            net[i] = 1
    return net

def derive(x, type):
    if (type == "linear"):
        return 1
    elif (type == "relu"):
        #x is activated neuron
        if (x > 0):
            return 1
        else:
            return 0
    elif (type == "sigmoid"):
        return x * (1-x)

def deriveSoftmax(output, k, target):
    idxTarget = target.index(max(target))
    if idxTarget == k:
        return -(1 - output[k])
    else:
        return output[k]

def loss(output, target, activation):
    if (activation == "softmax"):
        try:
            return (-1) * math.log(output[target.index(max(target))])
        except:
            return 0
    else:
        total = 0
        for i in range(len(output)):
            total += (output[i] - target[i]) ** 2
        return 0.5 * total


def classification(net, type):
    if (type == "linear"):
        return net
    elif (type == "sigmoid"):
        if (net > 0.5):
            return "Kelas Positif"
        else:
            return "Kelas Negatif"
    elif (type == "reLU"):
        return net
    elif (type == "softmax"):
        return net.index(max(net))

def countSSE(output, expected):
    res = 0
    for i in range(len(output)):
        temp = output[i][len(output)-1] - expected[i]
        res += temp**2
    return res

In [3]:
class ANN:
    def readInput(self, activation, layer, neuron_each_layer, inp, weight, target, learning_rate, batch_size, max_iter, error_threshold):
        self.activation = activation
        self.layer = layer 
        self.neuron_each_layer = neuron_each_layer
        self.inp = inp
        self.weight = weight
        self.target = target
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.error_threshold = error_threshold
        self.loss= []
        self.delta_weight = copy.deepcopy(self.weight)


    def readFile(self, file):
        f = open(file, "r")
        data = json.load(f)
        case = data["case"]

        self.activation = []
        for i in case["model"]["layers"]:
            self.activation.append(i['activation_function'])
        self.layer = len(case["model"]["layers"])
        self.neuron_each_layer = [case["model"]["input_size"]]
        for i in case["model"]["layers"]:
            self.neuron_each_layer.append(i['number_of_neurons'])
        self.inp = case["input"]
        self.weight = case["initial_weights"]
        self.target = case["target"]
        self.learning_rate = case["learning_parameters"]["learning_rate"]
        self.batch_size = case["learning_parameters"]["batch_size"]
        self.max_iter = case["learning_parameters"]["max_iteration"]
        self.error_threshold = case["learning_parameters"]["error_threshold"]
        
        expect = data["expect"]
        if ("final_weights" in expect):
            self.expected = expect["final_weights"]
        self.loss= []
        self.stopped_by = expect["stopped_by"]
        self.delta_weight = copy.deepcopy(self.weight)

    def generateWeight(self):
        for i in range(len(self.weight)):
            for j in range(len(self.weight[i])):
                for k in range(len(self.weight[i][j])):
                    self.weight[i][j][k] = random.randint(-100, 100) / 100 * 0.5

    def forwardProp(self, inp):
        instance = 0
        err = []
        for x in range(len(inp)):
            print("\nInstance:", inp[x])
            h = [inp[x]]
            for i in range(len(self.weight)):
                h.append([0 for j in range(self.neuron_each_layer[i + 1])])
                net = [0 for j in range(self.neuron_each_layer[i + 1])]
                for k in range(len(self.weight[i][0])):
                    net[k] += self.weight[i][0][k]
                for j in range(1, len(self.weight[i])):
                    for k in range(len(self.weight[i][j])):
                        net[k] += h[i][j - 1] * self.weight[i][j][k]
                
                if self.activation[i] == "softmax":
                    h[i+1] = eval(self.activation[i] + "({})".format(net))
                else:
                    for j in range(len(net)):
                        h[i + 1][j] = eval(self.activation[i] + "({})".format(net[j]))
            err_each = loss(h[len(h) - 1], self.target[x], self.activation[-1])
            err.append(err_each)
            print("Error:", err_each)
            print("Output:", h[len(h) - 1])
            print("Hidden Layer:", h)
            self.output.append(h[len(h) - 1])
            self.hidden_layer.append(h)
            instance += 1
        avg_error = np.average(err)
        print("Average of Error: ", avg_error)
        return avg_error

    def gradient(self, index, level, j, k, is_leaf):
        if (self.activation[level] == "softmax"):
            d_layer_net = deriveSoftmax(self.hidden_layer[index][level+1], k, self.target[index])
        else:
            d_layer_net = derive(self.hidden_layer[index][level+1][k], self.activation[level])
        if (is_leaf):
            if (j - 1 < 0):
                d_net_value = 1
            else:
                d_net_value = self.hidden_layer[index][level][j - 1]              
        else:
            d_net_value = self.weight[level][j][k] 
        if (level + 1 == len(self.hidden_layer[index]) - 1):
            d_E_o = (-1) * (self.target[index][k] - self.output[index][k])
            if (self.activation[level] != "softmax"):
                return d_E_o * d_layer_net * d_net_value
            else:
                return d_layer_net * d_net_value
        else:
            parent_gradient = 0
            for m in range(len(self.weight[level + 1][k+1])):
                parent_gradient += self.gradient(index, level + 1, k+1, m, False)
            return d_layer_net * d_net_value * parent_gradient

    def backwardProp(self, start, end, count):
        print("Initial weight:")
        print(np.array(self.weight))
        print("Epoch -", count+1)
        print("-------------------------------")
        err = self.forwardProp(self.inp[start:end])
        is_backward = True;
        if (err <= self.error_threshold):
            print("Stopped by error_treshold")
            is_backward = False;
        if is_backward:
            self.delta_weight_all = []
            for index in range(len(self.output)):
                self.delta_weight = copy.deepcopy(self.weight)
                for i in range(len(self.weight) - 1, -1, - 1):
                    for j in range(len(self.weight[i])):
                        for k in range(len(self.weight[i][j])):
                            self.delta_weight[i][j][k] = self.gradient(index, i, j, k, True)
                self.delta_weight_all.append(self.delta_weight)
            for i in range(len(self.delta_weight_all[0])):
                for j in range(len(self.delta_weight_all[0][i])):
                    for k in range(len(self.delta_weight_all[0][i][j])):
                        total_weight = 0 
                        for l in range(len(self.delta_weight_all)):
                            total_weight += self.delta_weight_all[l][i][j][k]
                        self.weight[i][j][k] -= self.learning_rate * total_weight
            print("Final Weight:")
            print(np.array(self.weight))
            print("\nEnd of epoch -", count + 1)
            print("-------------------------------")
            return True
        else:
            return False
    
    def mini_batch(self):
        isContinue = True
        count = 0
        while isContinue:
            self.output=[]
            self.hidden_layer=[]
            self.loss= []
            for i in range(0,len(self.inp),self.batch_size):
                isContinue = self.backwardProp(i, i+self.batch_size, count)
                if (not isContinue):
                    break
            count = count+1
            if (count >= self.max_iter):
                isContinue = False
            if (not isContinue):
                break
    
    def saveModel(self, file_name):
        model_dictionary = {
            "case": {
                "model": {
                    "input_size": self.neuron_each_layer[0] 
                },
                "input": self.inp,
                "weights": self.weight,
            },
            "expect": {
                "output": self.target,
                "max_sse": self.error_threshold
            }
        }
        model_dictionary["case"]["model"]["layers"] = []
        for idx, activation_function in enumerate(self.activation):
            model_dictionary["case"]["model"]["layers"].append({
                "number_of_neurons": self.neuron_each_layer[idx + 1],
                "activation_function": activation_function
            })
        json_object = json.dumps(model_dictionary, indent=4)
        with open(file_name, "w") as savefile:
            savefile.write(json_object)
    

In [4]:
file = "sigmoid_mini_batch_GD.JSON"

ann = ANN()
ann.readFile("test/{}".format(file))
ann.mini_batch()
ann.saveModel("output/{}".format(file))

  [-3.12903193 -7.93857454]]]
Epoch - 2450
-------------------------------

Instance: [1.0, 0.0]
Error: 0.2651427526719427
Output: [0.8262456510201048, 0.9465887678641521]
Hidden Layer: [[1.0, 0.0], [0.8262456510201048, 0.9465887678641521]]

Instance: [1.0, 1.0]
Error: 0.3426068941424451
Output: [0.17224742701861426, 0.006282213589645244]
Hidden Layer: [[1.0, 1.0], [0.17224742701861426, 0.006282213589645244]]
Average of Error:  0.3038748234071939
Final Weight:
[[[ 0.39716091  0.40264748]
  [ 1.16214742  2.47264135]
  [-3.13029247 -7.9395494 ]]]

End of epoch - 2450
-------------------------------
Initial weight:
[[[ 0.39716091  0.40264748]
  [ 1.16214742  2.47264135]
  [-3.13029247 -7.9395494 ]]]
Epoch - 2451
-------------------------------

Instance: [0.0, 0.0]
Error: 0.20427546035710836
Output: [0.5980053485648965, 0.5993235800362434]
Hidden Layer: [[0.0, 0.0], [0.5980053485648965, 0.5993235800362434]]

Instance: [0.0, 0.1]
Error: 0.1960774989781995
Output: [0.5210205194413424, 0.403

In [5]:
x = X.tolist()
nn = ANN()
activation = ['relu', 'relu', 'softmax']
layer = 4
neuron_each_layer = [4, 4, 4, 3]
inp = x
weight = [
    [
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
    ],
    [
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
        [0,0,0,0],
    ],
    [
        [0,0,0],
        [0,0,0],
        [0,0,0],
        [0,0,0],
        [0,0,0],
    ]
]

target = []
for t in y:
    if t == 0:
        target.append([1,0,0])
    elif t == 1:
        target.append([0,1,0])
    elif t == 2:
        target.append([0,0,1])
    else:
        print("Classnya lebih dari tiga")
learning_rate = 0.0001
batch_size = 10
max_iter = 100
error_threshold = 0.0001

nn.readInput(activation, layer, neuron_each_layer, inp, weight, target, learning_rate, batch_size, max_iter, error_threshold)
nn.generateWeight()
nn.mini_batch()
nn.saveModel("output/iris.json")

Instance: [6.6, 2.9, 4.6, 1.3]
Error: 0.7325607879717084
Output: [0.009600397179211814, 0.48067650208866325, 0.5097231007321249]
Hidden Layer: [[6.6, 2.9, 4.6, 1.3], [1.2821157830185252, 4.497728636765784, 0, 0], [1.6219369214833967, 0, 3.09727380929196, 0], [0.009600397179211814, 0.48067650208866325, 0.5097231007321249]]

Instance: [5.6, 2.7, 4.2, 1.3]
Error: 0.7437680769121015
Output: [0.017444422419811353, 0.4753194964691739, 0.5072360811110147]
Hidden Layer: [[5.6, 2.7, 4.2, 1.3], [0.9300647017275998, 3.8825676689863142, 0, 0], [1.375320629798665, 0, 2.8174288591767356, 0], [0.017444422419811353, 0.4753194964691739, 0.5072360811110147]]

Instance: [5.4, 3.4, 1.5, 0.4]
Error: 0.261237736159364
Output: [0.7700978177534475, 0.14765474886233398, 0.08224743338421853]
Hidden Layer: [[5.4, 3.4, 1.5, 0.4], [2.988802468240207, 1.1022957983401982, 0, 0], [0.04899022579602208, 0, 0, 0], [0.7700978177534475, 0.14765474886233398, 0.08224743338421853]]

Instance: [5.1, 3.4, 1.5, 0.2]
Error: 0.25

In [14]:
#MLP SKLEARN
print("MLP Sklearn")
clf = MLPClassifier(hidden_layer_sizes=(4, 4), activation='relu',
                    max_iter=100, batch_size=10, learning_rate_init=0.0001, tol=0.0001)
clf.fit(iris.data, iris.target)
pred = clf.predict(x)

print("Accuracy: " + str(metrics.accuracy_score(y, pred)))
print("F1: " + str(metrics.f1_score(y, pred, average='weighted')))
print("Confusion Matrix: ")

metrics.confusion_matrix(y, pred)

# Tugas B
f = open("output/iris.json", "r")
data = json.load(f)
case = data["case"]

# Input
inp = case["input"]

# Activation
activation = []
for i in case["model"]["layers"]:
    activation.append(i['activation_function'])

# Neuron
neuron = [case["model"]["input_size"]]
for i in case["model"]["layers"]:
    neuron.append(i['number_of_neurons'])

# Weight
weight = case['weights']

# Expected Output
expect = data["expect"]
eoutput = expect["output"]
sse = expect["max_sse"]
f.close()

instance = 0
output = []
for item in inp:
    h = [item]
    for i in range(len(weight)):
        h.append([0 for j in range(neuron[i + 1])])
        net = [0 for j in range(neuron[i + 1])]
        for k in range(len(weight[i][0])):
            net[k] += weight[i][0][k]
        for j in range(1, len(weight[i])):
            for k in range(len(weight[i][j])):
                net[k] += h[i][j - 1] * weight[i][j][k]
        
        if activation[i] == "softmax":
            h[i+1] = eval(activation[i] + "({})".format(net))
        else:
            for j in range(len(net)):
                h[i + 1][j] = eval(activation[i] + "({})".format(net[j]))
    output.append(h[len(h)-1])
    instance += 1

print("Class ANN")
true = []
pred = []
for i in range(len(output)):
    true.append(eoutput[i].index(max(eoutput[i])))
    pred.append(output[i].index(max(output[i])))
print("Accuracy:", metrics.accuracy_score(true, pred))
print("F1 Score:", metrics.f1_score(true, pred, average='weighted'))

MLP Sklearn
Accuracy: 0.6666666666666666
F1: 0.5555555555555555
Confusion Matrix: 
Class ANN
Accuracy: 0.7666666666666667
F1 Score: 0.7340930674264008
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
hiddenLayer = len(neuron)-2
print("x("+ str(neuron[0]) +")", end=" --> ")
for i in range(hiddenLayer):
    print("h"+str(i+1)+"("+ str(neuron[i+1]) +")", end=" --> ")
print("y("+ str(neuron[len(neuron)-1]) +")\n")

print("Weight:")
print("\tInput layer:")
if (hiddenLayer == 0):
    for i in range(int(neuron[1])):
        print("\t* b -> y("+str(i+1)+") = "+ str(weight[0][0][i]))
    for i in range(int(neuron[0])):
        for j in range(int(neuron[1])):
            print("\t* x("+ str(i+1) +") -> y("+ str(j+1)+") = "+ str(weight[0][i+1][j]))
else:
    for i in range(int(neuron[1])):
        print("\t* b -> h1("+str(i+1)+") = "+ str(weight[0][0][i]))
    for i in range(int(neuron[0])):
        for j in range(int(neuron[1])):
            print("\t* x("+ str(i+1) +") -> h1("+ str(j+1)+") = "+ str(weight[0][i+1][j]))

if hiddenLayer != 0:
    print("\n\tHidden Layer:")
    for i in range(hiddenLayer):
        if (i+1 == hiddenLayer):
            print("\th"+str(i+1))
            for m in range(int(neuron[i+2])):
                print("\t* b -> y("+ str(m+1) + ") = "+ str(weight[i+1][0][m]))
            for n in range(int(neuron[i+1])):
                for k in range(int(neuron[i+2])):
                    print("\t* h"+ str(i+1) + "(" + str(n+1) + ") -> y("+ str(k+1) +") = "+ str(weight[i+1][n+1][k]))
        else:
            print("\th"+str(i+1))
            for m in range(int(neuron[i+2])):
                print("\t* b -> h"+ str(i+2) + "(" + str(m+1)+") = "+ str(weight[i+1][0][m]))
            for n in range(int(neuron[i+1])):
                for k in range(int(neuron[i+2])):
                    print("\t* h"+ str(i+1) + "(" + str(n+1) +") -> h"+ str(i+2) + "(" + str(k+1)+") = "+ str(weight[i+1][n+1][k]))

print("\nActivation function:")
for i in range(len(activation)):
    if i == 0:
        if len(activation) == 1:
            print("x --> y = " + activation[i])
        else:
            print("x --> h1 = " + activation[i])
    elif i == len(activation)-1:
        print("h"+ str(i) +" --> y = " + activation[i])
    else:
        print("h"+ str(i) +" --> h"+ str(i+1) + " = " + activation[i])

x(4) --> h1(4) --> h2(4) --> y(3)

Weight:
	Input layer:
	* b -> h1(1) = -0.31813638492303814
	* b -> h1(2) = -0.5513254195908769
	* b -> h1(3) = 0.285
	* b -> h1(4) = 0.435
	* x(1) -> h1(1) = 0.499563682449436
	* x(1) -> h1(2) = 0.38498742570879496
	* x(1) -> h1(3) = -0.335
	* x(1) -> h1(4) = -0.18
	* x(2) -> h1(1) = 0.4757196459833055
	* x(2) -> h1(2) = -0.5089977344788427
	* x(2) -> h1(3) = -0.46
	* x(2) -> h1(4) = -0.395
	* x(3) -> h1(1) = -0.603557887195607
	* x(3) -> h1(2) = 0.8291198179338322
	* x(3) -> h1(3) = -0.2
	* x(3) -> h1(4) = -0.495
	* x(4) -> h1(1) = -0.22807308091946568
	* x(4) -> h1(2) = 0.13373149770526582
	* x(4) -> h1(3) = 0.475
	* x(4) -> h1(4) = 0.3

	Hidden Layer:
	h1
	* b -> h2(1) = -0.2551044323527735
	* b -> h2(2) = -0.275
	* b -> h2(3) = 0.07659305795152159
	* b -> h2(4) = -0.455
	* h1(1) -> h2(1) = -0.05780854136217735
	* h1(1) -> h2(2) = -0.425
	* h1(1) -> h2(3) = -0.7578941592397257
	* h1(1) -> h2(4) = 0.185
	* h1(2) -> h2(1) = 0.4357522916195159
	* h1(2

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e02db8a3-47d5-4c48-a321-8f1424eda79b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>